# Projeto 2 Recuperacao da Informacao

In [ ]:
#Parte referente aos métodos de processamento de consulta, considerando o modelo de espaço
#de vetores e método de comparação de ranqueamentos

#Autor: Marcus Vinicius Silva

In [1]:
from __future__ import division, unicode_literals

 
from time import time
import timeit
import codecs
import numpy as np
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import itertools as it

In [2]:
#essa funcao recebe o nome do arquivo html, e adiciona a representacao em texto dele na lista de strings

def parse_html(file_name, lista):
    
    f = codecs.open(file_name, 'r', 'utf-8', 'replace') #pode dar problema usar esse encoding, mas
    #a opcao replace ignora erros de caracters substituindo por ? na string, que no final das contas
    #vai ser desconsiderado do vocabulario
    
    
    
    with f as x_file:
        soup = BeautifulSoup(x_file.read())
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        
        script.decompose()
        
    # get text
    text = ' '.join(  soup.stripped_strings  )

    lista.append(text)

In [3]:
#essa funcao le todos os arquivos html para montar a base de dados

def ler_dados(lista):    
    
    
    PATH_FILE = './paginas_positivas/trainset' #lendo da pasta de paginas positivas
    i = 1
    limite = NUM_DOCS
    
    #ler paginas positivas
    while i < (limite + 1):
        file_name = PATH_FILE + ' (' +  str(i) + ').htm'  
        parse_html(file_name, lista)
        i += 1    


# Modelo espaço de vetores sem tf-idf

In [4]:
#lista dos documentos
NUM_DOCS = 50
lista_dados = [] #tam definido na funcao ler_dados
fator_norm = np.zeros(NUM_DOCS) #fator usado para normalizacao no cosine score
indice = []

#ler arquivos html e gerar lista de docs
ler_dados(lista_dados)
#calcular freq dos termos, representacao bag of words
vectorizer = CountVectorizer(strip_accents='unicode', max_df=0.9)
base_docs = vectorizer.fit_transform(lista_dados)


for i in range(NUM_DOCS):
    fator_norm[i] = len(base_docs[i].indices)
    

#Construcao do indice

#montando a cabeca dos postings
for chave in vectorizer.vocabulary_.keys():
    indice.append([chave,0, []]) #o segundo valor na verdade eh o doc_freq

#criando os postings
for post_head in indice:
    for i in range(np.shape(base_docs)[0]):
        coluna = vectorizer.vocabulary_[post_head[0]] #procura index do termo na base de dados
        #representada em bag of words
        if base_docs[i,coluna] != 0:
            post_head[2].append([i, base_docs[i,coluna]]) #i eh o id do doc e basedocs armazena a freq
            
    post_head[1] = NUM_DOCS/len(post_head[2]) #frequencia dos documentos que contem o termo

C:\Users\MarcusVinicius\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\MarcusVinicius\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [5]:
indice[5:10]

[['rio',
  1.6666666666666667,
  [[0, 3],
   [1, 1],
   [2, 1],
   [3, 1],
   [4, 1],
   [5, 1],
   [6, 1],
   [7, 3],
   [9, 1],
   [10, 1],
   [12, 1],
   [13, 1],
   [14, 3],
   [16, 1],
   [17, 8],
   [18, 1],
   [20, 3],
   [21, 1],
   [22, 1],
   [23, 6],
   [24, 1],
   [25, 1],
   [26, 8],
   [35, 3],
   [43, 1],
   [44, 3],
   [45, 1],
   [46, 3],
   [47, 3],
   [48, 8]]],
 ['8v',
  4.545454545454546,
  [[0, 2],
   [7, 3],
   [13, 1],
   [14, 3],
   [15, 1],
   [17, 3],
   [18, 3],
   [20, 3],
   [26, 4],
   [32, 2],
   [48, 3]]],
 ['flex',
  1.9230769230769231,
  [[0, 2],
   [1, 2],
   [2, 2],
   [4, 2],
   [7, 3],
   [8, 3],
   [11, 4],
   [12, 2],
   [13, 4],
   [14, 3],
   [15, 5],
   [16, 2],
   [17, 1],
   [20, 3],
   [23, 1],
   [24, 1],
   [25, 1],
   [26, 6],
   [27, 1],
   [32, 1],
   [33, 2],
   [35, 3],
   [44, 3],
   [47, 3],
   [48, 1],
   [49, 1]]],
 ['4p',
  2.9411764705882355,
  [[0, 2],
   [1, 2],
   [2, 2],
   [3, 2],
   [4, 2],
   [5, 2],
   [6, 2],
   [7, 3

In [7]:
#FUNCAO PARA CALCULAR RESULTADO RANQUEADO DA CONSULTA

#assume q como uma string split da consulta em texto livre
def cosine_sim(q, k):
    scores = np.zeros(NUM_DOCS)
    #vetor relativo a norma dos docs eh global
    for term in q:
        post_list_t = [item for item in indice if item[0] == term]
        
        if post_list_t: #pode ocorrer de o termo da pesquisa nao estar indexado
            post_list_t = post_list_t[0]
            for tupla in post_list_t[2]: #tupla 0 eh o id e tupla 1 do peso do termo no doc
                scores[tupla[0]] += post_list_t[1]*tupla[1]    #peso do termo na consulta vezes no doc 
        
    
    ind_scores = []
    
    for i in range(NUM_DOCS):
        scores[i] = scores[i] / (fator_norm[i]*len(q)) #consulta tambem deve ser normalizada
        ind_scores.append([scores[i], i])
    
    ind_scores.sort()
    
        
    
    return ind_scores[:-(k + 1):-1] #retornar k melhores resultados


In [8]:
consulta = 'ford ranger 2007'
resultado_consulta = cosine_sim(consulta.split(), 2)
resultado_consulta

[[0.015644813332674604, 38], [0.013588263588263589, 17]]

# Modelo espaço de vetores com tf-idf

In [72]:
from sklearn.feature_extraction.text import TfidfTransformer


tfidf_transformer = TfidfTransformer(smooth_idf=False)

base_docs2 = tfidf_transformer.fit_transform(base_docs)

#cria indice separado
indice2 = []

#montando a cabeca dos postings
for chave in vectorizer.vocabulary_.keys():
    indice2.append([chave,0, []]) #o segundo valor na verdade eh o doc_freq

#criando os postings
for post_head in indice:
    for i in range(np.shape(base_docs)[0]):
        coluna = vectorizer.vocabulary_[post_head[0]] #procura index do termo na base de dados
        #representada em bag of words
        if base_docs[i,coluna] != 0:
            post_head[2].append([i, base_docs2[i,coluna]]) #i eh o id do doc e basedocs armazena a freq
            
    post_head[1] = NUM_DOCS/len(post_head[2]) #frequencia dos documentos que contem o termo


# Comparando consultas

In [67]:
#funcao para calcular indice kendal tau
#assume receber dois resultados de consultas no formato [[peso, id]]
#assume resultados contendo os mesmos ids, possivelmente com rankings diferentes
def kendal_tau(res1, res2):
    
    lista_ids1 = [i[1] for i in res1]
    lista_ids2 = [i[1] for i in res2]
    
    
    pares1 = list(it.permutations(lista_ids1, 2))
    pares2 = list(it.permutations(lista_ids2, 2))
    pares_ajust1 = [] #vai eliminar alguns pares ordenados da permutacao original
    pares_ajust2 = []
    
    #dicionarios para armazenar valor do indice original do doc na sua resposta, acessando por id
    dic_indx1 = {}
    dic_indx2 = {}
    
    k = len(res1)
    
    for i, elem in enumerate(res1, start = 0):
        dic_indx1[elem[1]] = i #salva a posicao do doc no resultado 1, para acessar por id depois
    
    for j, elem in enumerate(res2, start = 0):
        dic_indx2[elem[1]] = j #salva a posicao do doc no resultado 1, para acessar por id depois
        
    
    for elem in pares1:
        if lista_ids1.index(elem[0]) < lista_ids1.index(elem[1]):
            pares_ajust1.append([elem[0], elem[1]])
            
    for elem in pares2:
        if lista_ids2.index(elem[0]) < lista_ids2.index(elem[1]):
            pares_ajust2.append([elem[0], elem[1]])    
    
    num_discor = 0 #numero de pares de ids discordantes entre os rankings
    #procurar pares discordantes
    for elem in pares_ajust1:
        if (dic_indx1[elem[0]] < dic_indx1[elem[1]]) and (dic_indx2[elem[0]] > dic_indx2[elem[1]]):
            num_discor += 1
        if (dic_indx1[elem[0]] > dic_indx1[elem[1]]) and (dic_indx2[elem[0]] < dic_indx2[elem[1]]):
            num_discor += 1
 
    for elem in pares_ajust2:
        if (dic_indx2[elem[0]] < dic_indx2[elem[1]]) and (dic_indx1[elem[0]] > dic_indx1[elem[1]]):
            num_discor += 1
        if (dic_indx2[elem[0]] > dic_indx2[elem[1]]) and (dic_indx1[elem[0]] < dic_indx1[elem[1]]):
            num_discor += 1
    
    
    return 1 - 2*num_discor/(k*(k-1))

In [68]:
res1 = [[50,123], [40,84], [30,56], [20,6] , [10,8]]
res2 = [[50,56], [40,123], [30,84], [20,8] , [10,6]]


teste_n = kendal_tau(res1, res2)

In [69]:
teste_n

0.4